In [2]:
!pip3 install polars
!pip3 install lxml

In [4]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import tostring
from IPython.display import FileLink, display
import os
import pandas as pd

In [5]:
filename = "/kaggle/input/so-posts/stackoverflow.com-Posts/Posts.xml"
batch_size = 5e5
limit = 120000

In [6]:
def append_batch_to_dataframes(batch, qa_df):
    q_df = pd.DataFrame([(row[:1] + row[2:]) for row in batch if str(row[1]) == "1"],
                        columns = [
                            'QuestionId',
                            'AcceptedAnswerId',
                            'QuestionScore',
                            'Question',
                            'Tags'
                        ])
    a_df = pd.DataFrame([(row[:1] + row[2:]) for row in batch if str(row[1]) == "2"],
                        columns = [
                            'AnswerId',
                            'QuestionId_',
                            'AnswerScore',
                            'Answer'
                        ])
    qa_df = pd.concat((qa_df,
                       pd.merge(
                           q_df,
                           a_df,
                           how='inner',
                           left_on='QuestionId',
                           right_on="QuestionId_"
                       ).drop(columns='QuestionId_')),
                       axis=0)
    return qa_df

In [ ]:
tree = ET.iterparse(filename, events=("start", "end"))
row_counter = 0
last_row_counter = 0
data_batch = []
qa_df = pd.DataFrame(columns=[
                        'QuestionId',
                        'AnswerId',
                        'AcceptedAnswerId',
                        'QuestionScore',
                        'AnswerScore',
                        'Question',
                        'Answer',
                        'Tags'
                    ])
for event, elem in tree:
    if str(elem.get("PostTypeId")) in ("1", "2"):
        if str(elem.get("PostTypeId")) == "1":
            if (
                'python' in elem.get("Tags")
                and "<img" not in elem.get("Body")
                and "<href" not in elem.get("Body")
                and int(elem.get("Score")) >= 20
                and elem.get("AcceptedAnswerId") is not None
            ):
                row_counter += 1
                data_batch.append(
                    [
                        elem.get("Id"),
                        elem.get("PostTypeId"),
                        elem.get("AcceptedAnswerId"),
                        elem.get("Score"),
                        elem.get("Body"),
                        elem.get("Tags").replace("<", "").replace(">", ",")[:-1],
                    ]
                )
        elif str(elem.get("PostTypeId")) == "2":
            if (
                "<img" not in elem.get("Body")
                and "<pre><code>" in elem.get("Body")
                and "<href" not in elem.get("Body")
            ):
                row_counter += 1
                data_batch.append(
                    [
                        elem.get("Id"),
                        elem.get("PostTypeId"),
                        elem.get("ParentId"),
                        elem.get("Score"),
                        elem.get("Body"),
                    ]
                )
        if row_counter % batch_size == 0 and last_row_counter != row_counter:
            last_row_counter = row_counter
            qa_df = append_batch_to_dataframes(data_batch, qa_df)
            data_batch = []
            if qa_df.shape[0] > 0:
                print(f'{qa_df.shape[0]} rows assembled')
        if qa_df.shape[0] >= limit:
            print("Dataset has been successfully assembled!")
            break
    elem.clear()

7709 rows assembled
11674 rows assembled
14482 rows assembled
16703 rows assembled
18721 rows assembled
20537 rows assembled
22201 rows assembled
23574 rows assembled
24936 rows assembled
26177 rows assembled
27372 rows assembled
28324 rows assembled
29109 rows assembled
29789 rows assembled
30502 rows assembled
31171 rows assembled
32019 rows assembled
32827 rows assembled
33589 rows assembled
34377 rows assembled
35149 rows assembled
35921 rows assembled
36541 rows assembled
37189 rows assembled
37776 rows assembled
38225 rows assembled
38658 rows assembled
39035 rows assembled
39418 rows assembled
39753 rows assembled
39979 rows assembled
40125 rows assembled


In [8]:
qa_df.to_csv("dataset.csv", index=False)
display(FileLink('dataset.csv'))

/kaggle/working/dataset.csv